In [64]:
import pandas as pd
import numpy as np
from collections import defaultdict


In [65]:
import numpy as np

user_embs = np.load("gcn_user_embeddings.npy")
item_embs = np.load("gcn_item_embeddings.npy")

print(user_embs.shape)  # (num_users, embedding_dim)
print(item_embs.shape)  # (num_items, embedding_dim)


(6040, 64)
(3706, 64)


In [66]:
item_embs.max()

np.float32(1.3287932)

In [67]:
import pandas as pd
from collections import defaultdict

train_df = pd.read_csv("train_split.csv")
test_df = pd.read_csv("test_split.csv")


In [68]:
train_df.head(10)

,userId,movieId,rating,timestamp,user,item
0,1,3186,1,978300019,0,2969
1,1,1270,1,978300055,0,1178
2,1,1721,1,978300055,0,1574
3,1,1022,1,978300055,0,957
4,1,2340,1,978300103,0,2147
5,1,1836,1,978300172,0,1658
6,1,3408,1,978300275,0,3177
7,1,2804,1,978300719,0,2599
8,1,1207,1,978300719,0,1117
9,1,260,1,978300760,0,253


In [69]:
train_dict = defaultdict(set)
test_dict = defaultdict(set)

for u, i in zip(train_df.userId, train_df.movieId):
    train_dict[u].add(i)

for u, i in zip(test_df.userId, test_df.movieId):
    test_dict[u].add(i)


In [70]:
import numpy as np

def recall_at_k(pred_items, gt_items, k):
    pred_items = pred_items[:k]
    return len(set(pred_items) & gt_items) / len(gt_items)

def ndcg_at_k(pred_items, gt_items, k):
    pred_items = pred_items[:k]
    dcg = 0.0
    for idx, item in enumerate(pred_items):
        if item in gt_items:
            dcg += 1 / np.log2(idx + 2)

    ideal_dcg = sum(1 / np.log2(i + 2) for i in range(min(len(gt_items), k)))
    return dcg / ideal_dcg if ideal_dcg > 0 else 0.0


In [71]:
def evaluate_lightgcn(user_embs, item_embs, train_dict, test_dict, K=20):
    recalls, ndcgs = [], []
    all_items = np.arange(item_embs.shape[0])

    for user in test_dict:
        u_emb = user_embs[user]

        # Exclude training interactions
        seen_items = train_dict[user]
        candidates = np.setdiff1d(all_items, list(seen_items), assume_unique=True)

        scores = np.dot(item_embs[candidates], u_emb)
        top_k_idx = np.argsort(scores)[-K:][::-1]
        top_k_items = candidates[top_k_idx]

        gt_items = test_dict[user]

        recalls.append(recall_at_k(top_k_items, gt_items, K))
        ndcgs.append(ndcg_at_k(top_k_items, gt_items, K))

    return np.mean(recalls), np.mean(ndcgs)


In [72]:
for K in [10, 20]:
    recall, ndcg = evaluate_lightgcn(
        user_embs,
        item_embs,
        train_dict,
        test_dict,
        K
    )
    print(f"Recall@{K}: {recall:.4f} | NDCG@{K}: {ndcg:.4f}")

IndexError: index 6040 is out of bounds for axis 0 with size 6040

In [73]:
all_users = sorted(train_df.userId.unique())
all_items = sorted(train_df.movieId.unique())

user2idx = {u: i for i, u in enumerate(all_users)}
item2idx = {i: j for j, i in enumerate(all_items)}


In [74]:
def remap(df):
    df = df.copy()
    df["userId"] = df["userId"].map(user2idx)
    df["movieId"] = df["movieId"].map(item2idx)
    return df.dropna().astype(int)

train_df = remap(train_df)
test_df = remap(test_df)


In [76]:
from collections import defaultdict

train_dict = defaultdict(set)
test_dict = defaultdict(set)

for u, i in zip(train_df.userId, train_df.movieId):
    train_dict[u].add(i)

for u, i in zip(test_df.userId, test_df.movieId):
    test_dict[u].add(i)


In [77]:
assert max(train_df.userId) < user_embs.shape[0]
assert max(train_df.movieId) < item_embs.shape[0]
assert user_embs.shape[1] == item_embs.shape[1]


In [78]:
for K in [10, 20]:
    recall, ndcg = evaluate_lightgcn(
        user_embs,
        item_embs,
        train_dict,
        test_dict,
        K
    )
    print(f"K={K} | Recall@{K}: {recall:.4f} | NDCG@{K}: {ndcg:.4f}")


K=10 | Recall@10: 0.0681 | NDCG@10: 0.1648
K=20 | Recall@20: 0.1186 | NDCG@20: 0.1692


In [79]:
def precision_at_k(pred_items, gt_items, k):
    pred_items = pred_items[:k]
    return len(set(pred_items) & gt_items) / k


def recall_at_k(pred_items, gt_items, k):
    pred_items = pred_items[:k]
    return len(set(pred_items) & gt_items) / len(gt_items)


def hitrate_at_k(pred_items, gt_items, k):
    pred_items = pred_items[:k]
    return 1.0 if len(set(pred_items) & gt_items) > 0 else 0.0


def ndcg_at_k(pred_items, gt_items, k):
    pred_items = pred_items[:k]
    dcg = 0.0
    for idx, item in enumerate(pred_items):
        if item in gt_items:
            dcg += 1 / np.log2(idx + 2)

    ideal_dcg = sum(
        1 / np.log2(i + 2)
        for i in range(min(len(gt_items), k))
    )
    return dcg / ideal_dcg if ideal_dcg > 0 else 0.0


In [80]:
def evaluate_lightgcn_full(
    user_embs,
    item_embs,
    train_dict,
    test_dict,
    K=20
):
    precisions, recalls, ndcgs, hitrates = [], [], [], []
    recommended_items_global = set()

    all_items = np.arange(item_embs.shape[0])

    for user in test_dict:
        u_emb = user_embs[user]

        seen_items = train_dict[user]
        candidates = np.setdiff1d(
            all_items,
            list(seen_items),
            assume_unique=True
        )

        scores = np.dot(item_embs[candidates], u_emb)
        top_k_idx = np.argsort(scores)[-K:][::-1]
        top_k_items = candidates[top_k_idx]

        gt_items = test_dict[user]

        recommended_items_global.update(top_k_items)

        precisions.append(precision_at_k(top_k_items, gt_items, K))
        recalls.append(recall_at_k(top_k_items, gt_items, K))
        ndcgs.append(ndcg_at_k(top_k_items, gt_items, K))
        hitrates.append(hitrate_at_k(top_k_items, gt_items, K))

    catalog_coverage = (
        len(recommended_items_global) / item_embs.shape[0]
    )

    return {
        "Precision@K": np.mean(precisions),
        "Recall@K": np.mean(recalls),
        "NDCG@K": np.mean(ndcgs),
        "HitRate@K": np.mean(hitrates),
        "CatalogCoverage@K": catalog_coverage
    }


In [81]:
for K in [10, 20]:
    metrics = evaluate_lightgcn_full(
        user_embs,
        item_embs,
        train_dict,
        test_dict,
        K
    )

    print(f"\nK = {K}")
    for m, v in metrics.items():
        print(f"{m}: {v:.4f}")



K = 10
Precision@K: 0.1518
Recall@K: 0.0681
NDCG@K: 0.1648
HitRate@K: 0.6525
CatalogCoverage@K: 0.2207

K = 20
Precision@K: 0.1371
Recall@K: 0.1186
NDCG@K: 0.1692
HitRate@K: 0.7972
CatalogCoverage@K: 0.2963


Adding BERT

In [82]:
item_bert_emb = np.load("bert_item_embeddings_final.npy")

In [83]:
item_bert_emb.shape

(3706, 64)

In [84]:
print(item_bert_emb.max())

0.4679058


In [85]:
ALPHA = 0.7  # tune later

item_final_emb = ALPHA * item_embs + (1 - ALPHA) * item_bert_emb


In [86]:
print(item_final_emb.shape)
print(item_embs.shape)
print(user_embs.shape)


(3706, 64)
(3706, 64)
(6040, 64)


In [87]:
def evaluate_recommender(
    user_embs,
    item_embs,
    train_dict,
    test_dict,
    K=20
):
    precisions, recalls, ndcgs, hitrates = [], [], [], []
    recommended_items_global = set()

    all_items = np.arange(item_embs.shape[0])

    for user in test_dict:
        u_emb = user_embs[user]

        seen_items = train_dict[user]
        candidates = np.setdiff1d(
            all_items,
            list(seen_items),
            assume_unique=True
        )

        scores = np.dot(item_embs[candidates], u_emb)
        top_k_idx = np.argsort(scores)[-K:][::-1]
        top_k_items = candidates[top_k_idx]

        gt_items = test_dict[user]
        recommended_items_global.update(top_k_items)

        precisions.append(len(set(top_k_items) & gt_items) / K)
        recalls.append(len(set(top_k_items) & gt_items) / len(gt_items))
        hitrates.append(1.0 if len(set(top_k_items) & gt_items) > 0 else 0.0)

        # NDCG
        dcg = 0.0
        for idx, item in enumerate(top_k_items):
            if item in gt_items:
                dcg += 1 / np.log2(idx + 2)

        idcg = sum(
            1 / np.log2(i + 2)
            for i in range(min(len(gt_items), K))
        )

        ndcgs.append(dcg / idcg if idcg > 0 else 0.0)

    catalog_coverage = (
        len(recommended_items_global) / item_embs.shape[0]
    )

    return {
        "Precision@K": np.mean(precisions),
        "Recall@K": np.mean(recalls),
        "NDCG@K": np.mean(ndcgs),
        "HitRate@K": np.mean(hitrates),
        "CatalogCoverage@K": catalog_coverage
    }


In [88]:
for K in [10, 20]:
    metrics = evaluate_recommender(
        user_embs=user_embs,
        item_embs=item_final_emb,
        train_dict=train_dict,
        test_dict=test_dict,
        K=K
    )

    print(f"\nFUSED EMBEDDINGS | K={K}")
    for m, v in metrics.items():
        print(f"{m}: {v:.4f}")



FUSED EMBEDDINGS | K=10
Precision@K: 0.1518
Recall@K: 0.0679
NDCG@K: 0.1645
HitRate@K: 0.6512
CatalogCoverage@K: 0.2178

FUSED EMBEDDINGS | K=20
Precision@K: 0.1370
Recall@K: 0.1184
NDCG@K: 0.1688
HitRate@K: 0.7977
CatalogCoverage@K: 0.2971


In [89]:
evaluate_recommender(user_embs, item_embs, train_dict, test_dict)


{'Precision@K': np.float64(0.13712748344370862),
 'Recall@K': np.float64(0.11864674503482625),
 'NDCG@K': np.float64(0.16920662410502893),
 'HitRate@K': np.float64(0.7971854304635762),
 'CatalogCoverage@K': 0.29627630868861304}

In [90]:
evaluate_recommender(user_embs, item_final_emb, train_dict, test_dict)


{'Precision@K': np.float64(0.13696192052980136),
 'Recall@K': np.float64(0.11836857429610727),
 'NDCG@K': np.float64(0.1687972986120715),
 'HitRate@K': np.float64(0.797682119205298),
 'CatalogCoverage@K': 0.29708580679978414}

In [91]:
evaluate_recommender(
    user_embs=np.zeros_like(user_embs),  # or averaged user embedding
    item_embs=item_bert_emb,
    train_dict=train_dict,
    test_dict=test_dict
)


{'Precision@K': np.float64(0.013418874172185432),
 'Recall@K': np.float64(0.009283788287184711),
 'NDCG@K': np.float64(0.020594745156321273),
 'HitRate@K': np.float64(0.1923841059602649),
 'CatalogCoverage@K': 0.028062601187263895}

In [92]:
import numpy as np

def l2_normalize(x):
    return x / np.linalg.norm(x, axis=1, keepdims=True)

item_gcn_norm = l2_normalize(item_embs)
item_bert_norm = l2_normalize(item_bert_emb)


In [93]:
import pandas as pd

alphas = np.arange(0.0, 1.01, 0.1)
results = []

for alpha in alphas:
    item_fused = alpha * item_gcn_norm + (1.0 - alpha) * item_bert_norm

    metrics = evaluate_recommender(
        user_embs=user_embs,
        item_embs=item_fused,
        train_dict=train_dict,
        test_dict=test_dict,
        K=20
    )

    results.append({
        "alpha": round(alpha, 2),
        "Precision@20": metrics["Precision@K"],
        "Recall@20": metrics["Recall@K"],
        "NDCG@20": metrics["NDCG@K"],
        "HitRate@20": metrics["HitRate@K"],
        "CatalogCoverage@20": metrics["CatalogCoverage@K"]
    })

df_results = pd.DataFrame(results)
df_results


,alpha,Precision@20,Recall@20,NDCG@20,HitRate@20,CatalogCoverage@20
0,0.0,0.004222,0.002477,0.004143,0.071689,0.232596
1,0.1,0.029288,0.021558,0.032381,0.360927,0.292229
2,0.2,0.050472,0.041867,0.058917,0.521523,0.387210
3,0.3,0.062144,0.054502,0.073808,0.583278,0.493794
4,0.4,0.069437,0.063226,0.082972,0.614404,0.591473
5,0.5,0.074363,0.069181,0.088980,0.628974,0.670264
6,0.6,0.078063,0.073857,0.094258,0.641391,0.731516
7,0.7,0.081167,0.076785,0.098347,0.644040,0.753373
8,0.8,0.083220,0.078811,0.101273,0.649834,0.759849
9,0.9,0.084073,0.079370,0.103010,0.645199,0.763357


In [94]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"

user_embs_t = torch.tensor(user_embs, dtype=torch.float32, device=device)
item_gcn_t = torch.tensor(item_embs, dtype=torch.float32, device=device)
item_bert_t = torch.tensor(item_bert_emb, dtype=torch.float32, device=device)

user_embs_t.requires_grad_(False)
item_gcn_t.requires_grad_(False)
item_bert_t.requires_grad_(False)


tensor([[-0.0037,  0.1535, -0.1533,  ...,  0.0017,  0.0974,  0.3149],
        [-0.0028,  0.1036, -0.1268,  ...,  0.0431,  0.1079,  0.3573],
        [ 0.0172,  0.1608, -0.1307,  ..., -0.0593,  0.0414,  0.2947],
        ...,
        [ 0.0702,  0.1134, -0.2216,  ..., -0.0540,  0.0652,  0.3976],
        [-0.0181,  0.1582, -0.1613,  ..., -0.0013,  0.0860,  0.2960],
        [ 0.0112,  0.0982, -0.0886,  ...,  0.0560,  0.1194,  0.2971]],
       device='cuda:0')

In [95]:
class ProjectionHybrid(nn.Module):
    def __init__(self, d_bert, d_gcn):
        super().__init__()
        self.W = nn.Linear(d_bert, d_gcn, bias=False)

    def forward(self, item_gcn, item_bert):
        return item_gcn + self.W(item_bert)


In [96]:
model = ProjectionHybrid(
    d_bert=item_bert_emb.shape[1],
    d_gcn=item_embs.shape[1]
).to(device)


In [97]:
def bpr_loss(u, pos, neg):
    pos_score = torch.sum(u * pos, dim=1)
    neg_score = torch.sum(u * neg, dim=1)
    return -torch.mean(F.logsigmoid(pos_score - neg_score))


In [98]:
import numpy as np
from collections import defaultdict

train_user_items = defaultdict(set)
for u, i in zip(train_df.userId, train_df.movieId):
    train_user_items[u].add(i)

num_items = item_embs.shape[0]

def sample_batch(batch_size=1024):
    users, pos_items, neg_items = [], [], []

    sampled_users = np.random.choice(list(train_user_items.keys()), batch_size)

    for u in sampled_users:
        pos = np.random.choice(list(train_user_items[u]))
        neg = np.random.randint(num_items)
        while neg in train_user_items[u]:
            neg = np.random.randint(num_items)

        users.append(u)
        pos_items.append(pos)
        neg_items.append(neg)

    return (
        torch.tensor(users, device=device),
        torch.tensor(pos_items, device=device),
        torch.tensor(neg_items, device=device)
    )


In [100]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 10
steps_per_epoch = 1000

model.train()

for epoch in range(epochs):
    total_loss = 0.0

    for _ in range(steps_per_epoch):
        u, pos, neg = sample_batch()

        u_emb = user_embs_t[u]

        pos_emb = model(item_gcn_t[pos], item_bert_t[pos])
        neg_emb = model(item_gcn_t[neg], item_bert_t[neg])

        loss = bpr_loss(u_emb, pos_emb, neg_emb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | BPR Loss: {total_loss/steps_per_epoch:.4f}")


KeyboardInterrupt: 

In [47]:
model.eval()
with torch.no_grad():
    item_final_emb = model(item_gcn_t, item_bert_t).cpu().numpy()

In [101]:
item_final_emb = np.load("item_final_emb.npy")

In [99]:
np.save("item_final_emb.npy", item_final_emb)

In [102]:
item_final_emb /= np.linalg.norm(item_final_emb, axis=1, keepdims=True)

In [103]:
metrics = evaluate_recommender(
    user_embs=user_embs,
    item_embs=item_final_emb,
    train_dict=train_dict,
    test_dict=test_dict,
    K=20
)

metrics


{'Precision@K': np.float64(0.08784768211920531),
 'Recall@K': np.float64(0.08216460450226688),
 'NDCG@K': np.float64(0.10875998968407581),
 'HitRate@K': np.float64(0.6564569536423841),
 'CatalogCoverage@K': 0.7622773880194279}

In [104]:
from collections import Counter

item_popularity = Counter(train_df.movieId)


In [105]:
# Sort items by popularity (ascending)
items_sorted = sorted(item_popularity.items(), key=lambda x: x[1])

num_items = len(items_sorted)
tail_cutoff = int(0.8 * num_items)

tail_items = set([item for item, _ in items_sorted[:tail_cutoff]])


In [106]:
print(f"Tail items count: {len(tail_items)} / {num_items}")

Tail items count: 2964 / 3706


In [107]:
cold_items = set(
    item for item, cnt in item_popularity.items() if cnt <= 5
)

print(f"Cold-start items count: {len(cold_items)}")

Cold-start items count: 439


In [108]:
from collections import defaultdict

def filter_test_dict(test_dict, target_items):
    filtered = defaultdict(set)
    for u, items in test_dict.items():
        filtered_items = items & target_items
        if len(filtered_items) > 0:
            filtered[u] = filtered_items
    return filtered

In [109]:
test_tail_dict = filter_test_dict(test_dict, tail_items)
test_cold_dict = filter_test_dict(test_dict, cold_items)

In [110]:
def evaluate_subset(
    user_embs,
    item_embs,
    train_dict,
    test_subset_dict,
    K=20
):
    return evaluate_recommender(
        user_embs=user_embs,
        item_embs=item_embs,
        train_dict=train_dict,
        test_dict=test_subset_dict,
        K=K
    )


In [111]:
print("TAIL ITEM EVALUATION (Recall@20)")

print("\nLightGCN:")
metrics_gcn_tail = evaluate_subset(
    user_embs, item_embs, train_dict, test_tail_dict
)
metrics_gcn_tail

print("\nProjection Hybrid:")
metrics_hybrid_tail = evaluate_subset(
    user_embs, item_final_emb, train_dict, test_tail_dict
)
metrics_hybrid_tail

TAIL ITEM EVALUATION (Recall@20)

LightGCN:

Projection Hybrid:


{'Precision@K': np.float64(0.05246332186198153),
 'Recall@K': np.float64(0.09849734796454053),
 'NDCG@K': np.float64(0.08416864827933437),
 'HitRate@K': np.float64(0.45408440499909436),
 'CatalogCoverage@K': 0.7622773880194279}

In [112]:
print("COLD-START ITEM EVALUATION (Recall@20)")

print("\nLightGCN:")
metrics_gcn_cold = evaluate_subset(
    user_embs, item_embs, train_dict, test_cold_dict
)
metrics_gcn_cold

print("\nProjection Hybrid:")
metrics_hybrid_cold = evaluate_subset(
    user_embs, item_final_emb, train_dict, test_cold_dict
)
metrics_hybrid_cold

COLD-START ITEM EVALUATION (Recall@20)

LightGCN:

Projection Hybrid:


{'Precision@K': np.float64(0.00013262599469496023),
 'Recall@K': np.float64(0.002652519893899204),
 'NDCG@K': np.float64(0.000696683116809532),
 'HitRate@K': np.float64(0.002652519893899204),
 'CatalogCoverage@K': 0.645979492714517}

In [113]:
import numpy as np
from collections import defaultdict

# average BERT embedding of items user interacted with
user_bert_profile = {}

for u, items in train_dict.items():
    if len(items) > 0:
        user_bert_profile[u] = item_bert_emb[list(items)].mean(axis=0)


In [114]:
def l2(x):
    return x / np.linalg.norm(x)

user_bert_profile = {u: l2(v) for u, v in user_bert_profile.items()}


In [115]:
def cosine(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

pseudo_pairs = []

TOP_K_SEM = 20   # small, conservative

for u, u_sem in user_bert_profile.items():
    scores = []
    for i in range(len(item_bert_emb)):
        if i not in train_dict[u]:
            s = cosine(u_sem, item_bert_emb[i])
            scores.append((i, s))

    scores.sort(key=lambda x: x[1], reverse=True)
    for i, _ in scores[:TOP_K_SEM]:
        pseudo_pairs.append((u, i))


In [116]:
def content_loss(u_emb, pos_emb, neg_emb):
    pos_score = torch.sum(u_emb * pos_emb, dim=1)
    neg_score = torch.sum(u_emb * neg_emb, dim=1)
    return -torch.mean(torch.log(torch.sigmoid(pos_score - neg_score)))


In [119]:
model.eval()
with torch.no_grad():
    item_final_emb = model(item_gcn_t, item_bert_t).cpu().numpy()


In [120]:
item_final_emb /= np.linalg.norm(item_final_emb, axis=1, keepdims=True)


In [121]:
import random
import torch

pseudo_pairs_list = list(pseudo_pairs)

def sample_pseudo_batch(batch_size=512):
    users, pos_items, neg_items = [], [], []
    for _ in range(batch_size):
        u, pos = random.choice(pseudo_pairs_list)
        neg = np.random.randint(len(item_gcn_emb))
        users.append(u)
        pos_items.append(pos)
        neg_items.append(neg)
    return (
        torch.tensor(users),
        torch.tensor(pos_items),
        torch.tensor(neg_items)
    )


In [122]:
def sample_pseudo_batch(batch_size=512):
    users, pos_items, neg_items = [], [], []

    for _ in range(batch_size):
        u, pos = random.choice(pseudo_pairs_list)
        neg = np.random.randint(num_items)

        users.append(u)
        pos_items.append(pos)
        neg_items.append(neg)

    return (
        torch.tensor(users, device=device),
        torch.tensor(pos_items, device=device),
        torch.tensor(neg_items, device=device)
    )


In [123]:
print(num_items)
print(item_gcn_t.shape)
print(item_bert_t.shape)


3706
torch.Size([3706, 64])
torch.Size([3706, 64])


In [124]:
model.eval()
with torch.no_grad():
    item_final_emb = model(item_gcn_t, item_bert_t).cpu().numpy()

item_final_emb /= np.linalg.norm(item_final_emb, axis=1, keepdims=True)


In [125]:
evaluate_recommender(user_embs, item_final_emb, train_dict, test_dict, K=20)


{'Precision@K': np.float64(0.09136589403973511),
 'Recall@K': np.float64(0.08578758074755703),
 'NDCG@K': np.float64(0.11221842695326636),
 'HitRate@K': np.float64(0.6713576158940397),
 'CatalogCoverage@K': 0.7137075013491635}

In [126]:
evaluate_subset(user_embs, item_final_emb, train_dict, test_tail_dict)


{'Precision@K': np.float64(0.045598623437783016),
 'Recall@K': np.float64(0.08245902750761391),
 'NDCG@K': np.float64(0.07132036322973857),
 'HitRate@K': np.float64(0.4048179677594639),
 'CatalogCoverage@K': 0.7137075013491635}

In [127]:
evaluate_subset(user_embs, item_final_emb, train_dict, test_cold_dict)


{'Precision@K': np.float64(0.00013262599469496023),
 'Recall@K': np.float64(0.002652519893899204),
 'NDCG@K': np.float64(0.0006137353134211118),
 'HitRate@K': np.float64(0.002652519893899204),
 'CatalogCoverage@K': 0.5928224500809498}